In [1]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer

import logging

logging.getLogger().setLevel(logging.INFO)

# Prepare Molecule Data

In [2]:
mol_file_name = './molecule-data/Aspirin.mol2'

mol_data = MoleculeData(mol_file_name, 'qmu')

INFO:root:parse mol2 file!


# Build QUBO model for molecule unfolding

In [3]:
model_param = {}
method = 'pre-calc'
if method == 'pre-calc':
    model_param['M'] = mol_data.bond_graph.rb_num
    model_param['D'] = 4
    model_param['A'] = 300
    model_param['hubo_qubo_val'] = 200
    
qmu_qubo = QMUQUBO(mol_data, method, **model_param)

INFO:root:pre-calculate for constructing molecule QUBO
INFO:root:['4+5']
INFO:root:elasped time for torsion group 4+5 : 2.0265579223632812e-07 min
INFO:root:['2+4']
INFO:root:elasped time for torsion group 2+4 : 7.549921671549479e-08 min
INFO:root:['1+2']
INFO:root:elasped time for torsion group 1+2 : 1.0728836059570313e-07 min
INFO:root:['10+11']
INFO:root:elasped time for torsion group 10+11 : 9.5367431640625e-08 min
INFO:root:['2+4', '4+5']
INFO:root:elasped time for torsion group 2+4,4+5 : 9.934107462565105e-08 min
INFO:root:['4+5', '10+11']
INFO:root:elasped time for torsion group 4+5,10+11 : 1.0331471761067709e-07 min
INFO:root:['1+2', '2+4']
INFO:root:elasped time for torsion group 1+2,2+4 : 9.139378865559896e-08 min
INFO:root:['1+2', '2+4', '4+5']
INFO:root:elasped time for torsion group 1+2,2+4,4+5 : 9.5367431640625e-08 min
INFO:root:['2+4', '4+5', '10+11']
INFO:root:elasped time for torsion group 2+4,4+5,10+11 : 9.5367431640625e-08 min
INFO:root:['1+2', '2+4', '4+5', '10+11']

# Optimize to find the configuration

In [4]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 1000

sa_optimizer = Annealer(qmu_qubo.qubo, method, **optimizer_param)

INFO:root:use simulated annealer from dimod


In [7]:
sa_optimizer.fit()
qa_optimizer.time_summary()

INFO:root:method dwave-qa complte time 0.23053516149520872
INFO:root:quantum annealer embed time 0.013261930147806803, optimize time 0.21727323134740192


In [5]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 1000
optimizer_param['bucket'] = f"amazon-braket-1a222675c751" # the name of the bucket
optimizer_param['prefix'] = "annealer-experiment" # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qmu_qubo.qubo, method, **optimizer_param)

INFO:root:use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


In [6]:
qa_optimizer.embed()
qa_optimizer.fit()
qa_optimizer.time_summary()

INFO:root:method dwave-qa complte time 0.23053516149520872
INFO:root:quantum annealer embed time 0.013261930147806803, optimize time 0.21727323134740192


# Post process for evaluation